In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
import pickle


def createModel_BASE():

    # CNN Model Architecture

    optimizer = tf.keras.optimizers.Adam(source_params['learning_rate'])
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(x_train_source.shape[1], x_train_source.shape[2])))
    model.add(tf.keras.layers.Conv1D(source_params['conv_filters_1'], 5, strides=1, padding='valid', activation='relu')) # 1D convolutional layer with 32 filters, a kernel size of 3, and a ReLU activation function
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Conv1D(source_params['conv_filters_2'], 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Conv1D(source_params['conv_filters_3'], 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(source_params['dense_units_1'], activation='relu'))
    model.add(tf.keras.layers.Dropout(source_params['dropout_rate_1']))
    model.add(tf.keras.layers.Dense(source_params['dense_units_2'], activation='relu'))
    model.add(tf.keras.layers.Dropout(source_params['dropout_rate_2']))
    model.add(tf.keras.layers.Dense(1))

    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

    model.summary()
    return model



def createModel_BASE_target():

    # LSTM Model Architecture

    optimizer = tf.keras.optimizers.Adam(target_params['learning_rate'])
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(x_train_target.shape[1], x_train_target.shape[2])))
    model.add(tf.keras.layers.Conv1D(target_params['conv_filters_1'], 5, strides=1, padding='valid', activation='relu')) # 1D convolutional layer with 32 filters, a kernel size of 3, and a ReLU activation function
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Conv1D(target_params['conv_filters_2'], 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Conv1D(target_params['conv_filters_3'], 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(target_params['dense_units_1'], activation='relu'))
    model.add(tf.keras.layers.Dropout(target_params['dropout_rate_1']))
    model.add(tf.keras.layers.Dense(target_params['dense_units_2'], activation='relu'))
    model.add(tf.keras.layers.Dropout(target_params['dropout_rate_2']))
    model.add(tf.keras.layers.Dense(1))

    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

    model.summary()
    return model



def createModel_BASE_target_2():

    # LSTM Model Architecture


    optimizer = tf.keras.optimizers.Adam(target_2_params['learning_rate'])
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(x_train_target_2.shape[1], x_train_target_2.shape[2])))
    model.add(tf.keras.layers.Conv1D(target_2_params['conv_filters_1'], 5, strides=1, padding='valid', activation='relu')) # 1D convolutional layer with 32 filters, a kernel size of 3, and a ReLU activation function
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Conv1D(target_2_params['conv_filters_2'], 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Conv1D(target_2_params['conv_filters_3'], 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(target_2_params['dense_units_1'], activation='relu'))
    model.add(tf.keras.layers.Dropout(target_2_params['dropout_rate_1']))
    model.add(tf.keras.layers.Dense(target_2_params['dense_units_2'], activation='relu'))
    model.add(tf.keras.layers.Dropout(target_2_params['dropout_rate_2']))
    model.add(tf.keras.layers.Dense(1))

    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

    model.summary()
    return model



def createModel_TL1(domain):

    model = tf.keras.models.load_model('Models/CNN/12M/BASE models/EC_BASE_source.keras')

    optimizer = tf.keras.optimizers.Adam(source_params['learning_rate'])
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])
    model.summary()

    return model



def createModel_TL2(domain):

    model = tf.keras.models.load_model('Models/CNN/12M/BASE models/EC_BASE_source.keras')
    model.layers[7].trainable = False
    model.layers[9].trainable = False

    #Compile model
    optimizer = tf.keras.optimizers.Adam(source_params['learning_rate'])
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])
    model.summary()

    return model



def createModel_TL3(domain):

    # LSTM Model Architecture

    base_model = tf.keras.models.load_model('Models/CNN/12M/BASE models/EC_BASE_source.keras')

    input_shape = base_model.input_shape[1:]

    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv1D(source_params['conv_filters_1'], 5, strides=1, padding='valid', activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Conv1D(source_params['conv_filters_2'], 3, activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Conv1D(source_params['conv_filters_3'], 3, activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Flatten()(x)

    for i in range(7,12):
        x = base_model.layers[i](x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.layers[7].trainable = False
    model.layers[9].trainable = False

    #Compile model
    optimizer = tf.keras.optimizers.Adam(source_params['learning_rate'])
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])
    model.summary()

    return model



def fit_model_plus_stats_BASE(model, x_train, y_train, domain, type_):

    # Early stopping property
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    if type_ == '6M':
        dir_save = 'Models/CNN/6M/BASE models/'
    elif type_ == '12M':
        dir_save = 'Models/CNN/12M/BASE models/'

    if domain == 'source':

        # Fit the model
        history = model.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=source_params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Save the model
        model.save('Models/CNN/12M/BASE models/EC_BASE_source.keras')

        #Load the model
        model = tf.keras.models.load_model('Models/CNN/12M/BASE models/EC_BASE_source.keras')

    elif domain == 'target':

        # Fit the model
        history = model.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=target_params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Save the model
        model.save( dir_save + 'EC_BASE_target.keras')

        #Load the model
        model = tf.keras.models.load_model(dir_save + 'EC_BASE_target.keras')

    elif domain == 'target_2':

        # Fit the model
        history = model.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=target_2_params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Save the model
        model.save( dir_save + 'EC_BASE_target_2.keras')

        #Load the model
        model = tf.keras.models.load_model(dir_save + 'EC_BASE_target_2.keras')

    # summarize history for MAE and MSE
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('Model MSE')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()



def fit_model_plus_stats_TL1(model, x_train, y_train, domain, type_):

    # Early stopping property
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6)

    if type_ == '6M':
        dir_save = 'Models/CNN/6M/BASE models/'
    elif type_ == '12M':
        dir_save = 'Models/CNN/12M/BASE models/'

    if domain == 'target':

        params = target_params

        # Fit the model
        history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Save the model
        model.save( dir_save + 'EC_TL1_target.keras')

        #Load the model
        model = tf.keras.models.load_model(dir_save + 'EC_TL1_target.keras')

    elif domain == 'target_2':

        params = target_2_params

        # Fit the model
        history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Save the model
        model.save( dir_save + 'EC_TL1_target_2.keras')

        #Load the model
        model = tf.keras.models.load_model(dir_save + 'EC_TL1_target_2.keras')

    # summarize history for MAE and MSE
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('Model MSE')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()



def fit_model_plus_stats_TL2(model, x_train, y_train, domain, type_):

    # Early stopping property
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6)

    if type_ == '6M':
        dir_save = 'Models/CNN/6M/BASE models/'
    elif type_ == '12M':
        dir_save = 'Models/CNN/12M/BASE models/'

    if domain == 'target':

        params = target_params

        # Fit the model
        history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Save the model
        model.save( dir_save + 'EC_TL2_target.keras')

        #Load the model
        model = tf.keras.models.load_model(dir_save + 'EC_TL2_target.keras')

    elif domain == 'target_2':

        params = target_2_params

        # Fit the model
        history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Save the model
        model.save( dir_save + 'EC_TL2_target_2.keras')

        #Load the model
        model = tf.keras.models.load_model(dir_save + 'EC_TL2_target_2.keras')

    # summarize history for MAE and MSE
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('Model MSE')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()



def fit_model_plus_stats_TL3(model, x_train, y_train, domain, type_):

    # Early stopping property
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    if type_ == '6M':
        dir_save = 'Models/CNN/6M/BASE models/'
    elif type_ == '12M':
        dir_save = 'Models/CNN/12M/BASE models/'

    if domain == 'target':

        params = target_params

        # Fit the model
        history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Fine Tune Model
        model.trainable = True
        model.layers[8].trainable = True
        model.layers[10].trainable = True

        optimizer = tf.keras.optimizers.Adam(source_params['learning_rate']*0.7) # low to prevent overfitting
        model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

        history = model.fit(x_train, y_train, epochs=20, validation_split=0.2, batch_size=params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Save the model
        model.save( dir_save + 'EC_TL3_target.keras')

        #Load the model
        model = tf.keras.models.load_model(dir_save + 'EC_TL3_target.keras')

    elif domain == 'target_2':

        params = target_2_params

        # Fit the model
        history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Fine Tune Model
        model.trainable = True
        model.layers[7].trainable = True
        model.layers[9].trainable = True
        optimizer = tf.keras.optimizers.Adam(source_params['learning_rate']*0.7) # low to prevent overfitting
        model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

        history = model.fit(x_train, y_train, epochs=20, validation_split=0.2, batch_size=params['batch_size'], verbose=1, shuffle=True, callbacks=[es]).history

        # Save the model
        model.save( dir_save + 'EC_TL3_target_2.keras')

        #Load the model
        model = tf.keras.models.load_model(dir_save + 'EC_TL3_target_2.keras')

    # summarize history for MAE and MSE
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('Model MSE')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()



def predict_model(model, x_test):

    y_pred = model.predict(x_test)

    return(y_pred)


def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res


def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred))))



def model_fit_predict_results(mode, type_):

    metrics_unscaled = []

    if mode == 'BASE_source':

        scaler = scaler_source
        y_test = y_test_source[:-24]
        sqm = sqm_source

        model = createModel_BASE()

        fit_model_plus_stats_BASE(model, x_train_source, y_train_source, 'source', type_)

        predictions_scaled = predict_model(model, x_test_source)


    elif mode == 'BASE_target' or mode == 'TL1_target' or mode == 'TL2_target' or mode == 'TL3_target':

        scaler = scaler_target
        y_test = y_test_target[:-24]
        sqm = sqm_target


        if mode == 'BASE_target':
            model = createModel_BASE_target()
            fit_model_plus_stats_BASE(model, x_train_target, y_train_target, 'target', type_)

        elif mode == 'TL1_target':

            model = createModel_TL1('target')
            fit_model_plus_stats_TL1(model, x_train_target, y_train_target, 'target', type_)

        elif mode == 'TL2_target':

            model = createModel_TL2('target')
            fit_model_plus_stats_TL2(model, x_train_target, y_train_target, 'target', type_)

        elif mode == 'TL3_target':

            model = createModel_TL3('target')
            fit_model_plus_stats_TL3(model, x_train_target, y_train_target, 'target', type_)

        predictions_scaled = predict_model(model, x_test_target)

    elif mode == 'BASE_target_2' or mode == 'TL1_target_2' or mode == 'TL2_target_2' or mode == 'TL3_target_2':

        scaler = scaler_target_2
        y_test = y_test_target_2[:-24]
        sqm = sqm_target_2

        if mode == 'BASE_target_2':
            model = createModel_BASE_target_2()
            fit_model_plus_stats_BASE(model, x_train_target_2, y_train_target_2, 'target_2', type_)

        elif mode == 'TL1_target_2':

            model = createModel_TL1('target_2')
            fit_model_plus_stats_TL1(model, x_train_target_2, y_train_target_2, 'target_2', type_)

        elif mode == 'TL2_target_2':

            model = createModel_TL2('target_2')
            fit_model_plus_stats_TL2(model, x_train_target_2, y_train_target_2, 'target_2', type_)

        elif mode == 'TL3_target_2':

            model = createModel_TL3('target_2')
            fit_model_plus_stats_TL3(model, x_train_target_2, y_train_target_2, 'target_2', type_)

        predictions_scaled = predict_model(model, x_test_target_2)

    predictions_scaled = predictions_scaled[:-24]

    rmse_scaled = np.sqrt(mean_squared_error(y_test, predictions_scaled))
    mae_scaled = mean_absolute_error(y_test, predictions_scaled)
    mape_scaled = mean_absolute_percentage_error(y_test, predictions_scaled)
    r2_score_scaled = r2_score(y_test, predictions_scaled)

    print('Test Scaled RMSE: {}'.format(rmse_scaled))
    print('Test Scaled MAE: {}'.format(mae_scaled))
    print('Test Scaled R2 Score: ',r2_score_scaled)
    print('Test Scaled MAPE: ',mape_scaled)

    print()

    predictions_unscaled = (scaler.inverse_transform(predictions_scaled))*(sqm/100)
    true_measurements_unscaled = (scaler.inverse_transform(y_test))*(sqm/100)

    rmse_unscaled = np.sqrt(mean_squared_error(true_measurements_unscaled, predictions_unscaled))
    mae_unscaled = mean_absolute_error(true_measurements_unscaled, predictions_unscaled)
    mape_unscaled = mean_absolute_percentage_error(true_measurements_unscaled, predictions_unscaled)
    r2_score_unscaled = r2_score(true_measurements_unscaled, predictions_unscaled)*100

    metrics_unscaled.append(rmse_unscaled)
    metrics_unscaled.append(mae_unscaled)
    metrics_unscaled.append(mape_unscaled)
    metrics_unscaled.append(r2_score_unscaled)

    print('Test Unscaled RMSE: {}'.format(rmse_unscaled))
    print('Test Unscaled MAE: {}'.format(mae_unscaled))
    print('Test Unscaled R2 Score: ',r2_score_unscaled)
    print('Test Unscaled MAPE: ',mape_unscaled)

    return predictions_unscaled, metrics_unscaled


#------------- MAKE PREDICTIONS FOR 12M DATASETS ---------------#

with open('Domain sqms/sqm_source.json', 'r') as f:
    sqm_source = json.load(f)

with open('Domain sqms/sqm_target.json', 'r') as f:
    sqm_target = json.load(f)

with open('Domain sqms/sqm_target_2.json', 'r') as f:
    sqm_target_2 = json.load(f)


scaler_source = joblib.load('Scalers/12M/source_scaler.pkl')
scaler_target = joblib.load('Scalers/12M/target_scaler.pkl')
scaler_target_2 = joblib.load('Scalers/12M/target_2_scaler.pkl')


x_train_source = np.load('Data/Preprocessed data/12M/source_train_x.npy')
y_train_source = np.load('Data/Preprocessed data/12M/source_train_y.npy')
x_test_source = np.load('Data/Preprocessed data/12M/source_test_x.npy')
y_test_source = np.load('Data/Preprocessed data/12M/source_test_y.npy')
x_train_target = np.load('Data/Preprocessed data/12M/target_train_x.npy')
y_train_target = np.load('Data/Preprocessed data/12M/target_train_y.npy')
x_test_target = np.load('Data/Preprocessed data/12M/target_test_x.npy')
y_test_target = np.load('Data/Preprocessed data/12M/target_test_y.npy')
x_train_target_2 = np.load('Data/Preprocessed data/12M/target_2_train_x.npy')
y_train_target_2 = np.load('Data/Preprocessed data/12M/target_2_train_y.npy')
x_test_target_2 = np.load('Data/Preprocessed data/12M/target_2_test_x.npy')
y_test_target_2 = np.load('Data/Preprocessed data/12M/target_2_test_y.npy')


with open('Models/CNN/12M/Tuned Hyperparameters/source_params.json', 'r') as f:
    source_params = json.load(f)

with open('Models/CNN/12M/Tuned Hyperparameters/target_params.json', 'r') as f1:
    target_params = json.load(f1)

with open('Models/CNN/12M/Tuned Hyperparameters/target_2_params.json', 'r') as f2:
    target_2_params = json.load(f2)


#BASE MODEL PREDICTIONS FOR SOURCE DOMAIN DATASET

predictions_base_source, metrics_base_source = model_fit_predict_results('BASE_source', '12M')

np.save('Models/CNN/12M/Predictions/predictions_base_source.npy', predictions_base_source)
with open('Models/CNN/12M/Metrics/metrics_base_source.pkl', 'wb') as file:
    pickle.dump(metrics_base_source, file)


#BASE MODEL PREDICTIONS FOR TARGET DOMAIN DATASET

predictions_base_target, metrics_base_target = model_fit_predict_results('BASE_target', '12M')

np.save('Models/CNN/12M/Predictions/predictions_base_target.npy', predictions_base_target)
with open('Models/CNN/12M/Metrics/metrics_base_target.pkl', 'wb') as file:
    pickle.dump(metrics_base_target, file)

#BASE MODEL PREDICTIONS FOR TARGET DOMAIN 2 DATASET

predictions_base_target_2, metrics_base_target_2 = model_fit_predict_results('BASE_target_2', '12M')

np.save('Models/CNN/12M/Predictions/predictions_base_target_2.npy', predictions_base_target_2)
with open('Models/CNN/12M/Metrics/metrics_base_target_2.pkl', 'wb') as file:
    pickle.dump(metrics_base_target_2, file)

#TL1 technique

predictions_TL1_target, metrics_TL1_target = model_fit_predict_results('TL1_target', '12M')

np.save('Models/CNN/12M/Predictions/predictions_TL1_target.npy', predictions_TL1_target)
with open('Models/CNN/12M/Metrics/metrics_TL1_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL1_target, file)

predictions_TL1_target_2, metrics_TL1_target_2 = model_fit_predict_results('TL1_target_2', '12M')

np.save('Models/CNN/12M/Predictions/predictions_TL1_target_2.npy', predictions_TL1_target_2)
with open('Models/CNN/12M/Metrics/metrics_TL1_target_2.pkl', 'wb') as file:
    pickle.dump(metrics_TL1_target_2, file)


#TL2 technique

predictions_TL2_target, metrics_TL2_target = model_fit_predict_results('TL2_target', '12M')

np.save('Models/CNN/12M/Predictions/predictions_TL2_target.npy', predictions_TL2_target)
with open('Models/CNN/12M/Metrics/metrics_TL2_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL2_target, file)

predictions_TL2_target_2, metrics_TL2_target_2 = model_fit_predict_results('TL2_target_2', '12M')

np.save('Models/CNN/12M/Predictions/predictions_TL2_target_2.npy', predictions_TL2_target_2)
with open('Models/CNN/12M/Metrics/metrics_TL2_target_2.pkl', 'wb') as file:
    pickle.dump(metrics_TL2_target_2, file)


#TL3 technique

predictions_TL3_target, metrics_TL3_target = model_fit_predict_results('TL3_target', '12M')

np.save('Models/CNN/12M/Predictions/predictions_TL3_target.npy', predictions_TL3_target)
with open('Models/CNN/12M/Metrics/metrics_TL3_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL3_target, file)

predictions_TL3_target_2, metrics_TL3_target_2 = model_fit_predict_results('TL3_target_2', '12M')

np.save('Models/CNN/12M/Predictions/predictions_TL3_target_2.npy', predictions_TL3_target_2)
with open('Models/CNN/12M/Metrics/metrics_TL3_target_2.pkl', 'wb') as file:
    pickle.dump(metrics_TL3_target_2, file)


#------------- MAKE PREDICTIONS FOR 6M DATASETS ---------------#


with open('Domain sqms/sqm_target.json', 'r') as f:
    sqm_target = json.load(f)


scaler_target = joblib.load('Scalers/6M/target_scaler.pkl')


x_train_target = np.load('Data/Preprocessed data/6M/target_train_x.npy')
y_train_target = np.load('Data/Preprocessed data/6M/target_train_y.npy')
x_test_target = np.load('Data/Preprocessed data/6M/target_test_x.npy')
y_test_target = np.load('Data/Preprocessed data/6M/target_test_y.npy')


with open('Models/CNN/6M/Tuned Hyperparameters/target_params.json', 'r') as f1:
    target_params = json.load(f1)


#BASE MODEL PREDICTIONS FOR TARGET DOMAIN DATASET

predictions_base_target, metrics_base_target = model_fit_predict_results('BASE_target', '6M')

np.save('Models/CNN/6M/Predictions/predictions_base_target.npy', predictions_base_target)
with open('Models/CNN/6M/Metrics/metrics_base_target.pkl', 'wb') as file:
    pickle.dump(metrics_base_target, file)


#TL1 technique

predictions_TL1_target, metrics_TL1_target = model_fit_predict_results('TL1_target', '6M')

np.save('Models/CNN/6M/Predictions/predictions_TL1_target.npy', predictions_TL1_target)
with open('Models/CNN/6M/Metrics/metrics_TL1_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL1_target, file)


#TL2 technique

predictions_TL2_target, metrics_TL2_target = model_fit_predict_results('TL2_target', '6M')

np.save('Models/CNN/6M/Predictions/predictions_TL2_target.npy', predictions_TL2_target)
with open('Models/CNN/6M/Metrics/metrics_TL2_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL2_target, file)


#TL3 technique

predictions_TL3_target, metrics_TL3_target = model_fit_predict_results('TL3_target', '6M')

np.save('Models/CNN/6M/Predictions/predictions_TL3_target.npy', predictions_TL3_target)
with open('Models/CNN/6M/Metrics/metrics_TL3_target.pkl', 'wb') as file:
    pickle.dump(metrics_TL3_target, file)
